In [4]:
# Imports
import os
import json
from typing import Tuple
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [5]:
# Load environment variables
load_dotenv(override=True)

True

In [6]:
# Show key prefixes (optional sanity check)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

for name, value in [
    ("OpenAI", openai_api_key),
    ("Anthropic", anthropic_api_key),
    ("Google", google_api_key),
    ("DeepSeek", deepseek_api_key),
    ("Groq", groq_api_key),
]:
    if value:
        print(f"{name} API Key exists and begins {value[:8]}")
    else:
        print(f"{name} API Key not set (optional for some)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-a
Google API Key exists and begins AIzaSyDJ
DeepSeek API Key exists and begins sk-997f5
Groq API Key exists and begins gsk_rI8B


In [7]:
# Base idea prompt
base_prompt = """
Please come up with an idea for an app that I can build using Agentic AI.
The idea must:
- Be buildable for Android, iPhone, and Web
- Be highly profitable and easy to monetize
- Attract a niche but abundant audience
- Avoid heavy custom back-end systems (prefer existing APIs)
- Avoid complex legal/regulatory exposure

Return only the idea plus concise arguments for how it meets each criterion.
"""

In [9]:
# Helper functions
openai_client = OpenAI()
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gpt-5-mini"

messages = [{"role": "user", "content": base_prompt}]

response = openai_client.chat.completions.create(
    model="gpt-5-mini",
    messages=messages )

answers = []
answer = response.choices[0].message.content
answers.append(answer)

def build_qa_prompt(idea: str) -> str:
    return f"""
You are evaluating an app idea. Score each of the following criteria 0 or 1 (integer only):
1) clarity
2) accuracy
3) strength of arguments
4) can be built entirely using Agentic AI
5) can be built on Android, Web, and iPhone
6) highly profitable
7) attracts a niche but abundant audience
8) easy to monetize
9) avoids custom extensive back-end systems; can rely on existing APIs
10) avoids overly complex legal/regulatory clearance

Return JSON only, with schema: {{"results": ["<total_score_0-10>", "<failed criteria or 'Met all criteria'>"]}}.
Keep the second field short (<= 60 words). No markdown, no extra text.

Idea to evaluate:
{idea}
"""

def parse_qa_response(text: str) -> Tuple[int, str]:
    try:
        data = json.loads(text)
        results = data.get("results") or []
        score_raw = results[0] if len(results) > 0 else None
        feedback = results[1] if len(results) > 1 else ""
        score = int(str(score_raw))
        return score, str(feedback)
    except Exception as exc:
        # On parse failure, treat as zero score and pass raw text back
        return 0, f"QA parse error: {exc}; raw: {text}"

def generate_idea(prompt: str) -> str:
    resp = openai_client.chat.completions.create(
    model="gpt-5-mini",
    messages=[{"role": "user", "content": prompt}],
    )
    return resp.choices[0].message.content

def evaluate_idea(idea: str) -> Tuple[int, str, str]:
    qa_prompt = build_qa_prompt(idea)
    resp = gemini.chat.completions.create(
        model="gemini-2.5-flash",
        messages=[{"role": "user", "content": qa_prompt}],
    )
    qa_text = resp.choices[0].message.content
    score, feedback = parse_qa_response(qa_text)
    return score, feedback, qa_text

In [10]:
# Refinement loop: iterate until score == 10 or 10 iterations reached
max_iterations = 10
prompt_for_idea = base_prompt
idea_history = []
qa_history = []
best_idea = None
best_score = -1

for i in range(1, max_iterations + 1):
    print(f"--- Iteration {i} ---")
    idea = generate_idea(prompt_for_idea)
    print("Idea:", idea)

    score, feedback, raw_qa = evaluate_idea(idea)
    print(f"QA score: {score}")
    print(f"QA feedback: {feedback}")

    idea_history.append(idea)
    qa_history.append({"iteration": i, "score": score, "feedback": feedback, "raw": raw_qa})

    if score > best_score:
        best_score = score
        best_idea = idea

    if score == 10:
        print("Perfect score reached. Stopping.")
        break

    # Prepare next prompt using feedback
    prompt_for_idea = base_prompt + f"Improve the previous idea using this QA feedback (fix deficiencies without bloating scope): {feedback} Return only the revised idea plus concise arguments."

print("=== Summary ===")
print(f"Best score: {best_score}")
print("Best idea:", best_idea)
print("QA trail:")
for entry in qa_history:
    print(f"Iter {entry['iteration']}: score={entry['score']} feedback={entry['feedback']}")

--- Iteration 1 ---
Idea: Idea: "LaunchGenie" — an agentic AI assistant that autonomously plans and runs pre-launch and post-launch marketing campaigns for indie/self-published authors.

How it works (brief): the user supplies book metadata (blurb, genre, target audience, launch date), content assets (excerpt, cover, trailer), and goals (preorders, mailing-list signups, Goodreads reviews). An agentic AI builds a step-by-step launch plan, drafts email sequences, social posts, ARC outreach messages, designs a giveaway plan, schedules posts and emails, and then autonomously executes and adapts the campaign (via integrations), reporting simple KPIs and recommending next steps.

How it meets your constraints

- Buildable for Android, iPhone, and Web
  - Frontends: single codebase (React Native + React Web) for mobile and web.
  - Light orchestration serverless functions (AWS Lambda / Vercel) to run agent tasks and coordinate APIs — no monolithic backend required.

- Highly profitable and ea

In [11]:
display(Markdown(best_idea))

Idea: "SessionForge" — an agentic-AI built marketplace and scheduling hub for tabletop RPG (TTRPG) Game Masters (GMs) and players to find, book, schedule, and manage paid or free one-shot and campaign sessions.

Core features (one line): profile + availability pages for GMs, searchable session listings, per-session booking & payments, calendar sync, chat/announcements, review/rating, and featured listings.

Why this revised idea meets each requirement

- Buildable for Android, iPhone, and Web
  - Implement as a PWA for Web + mobile; use no-code app builders (Glide/Adalo/FlutterFlow/Webflow) that provide native wrappers and app-store export APIs. Agentic AI agents can programmatically create the UI/flows in those builders via their APIs and publish web + native builds — no hand-coded frontends required.

- Can be built entirely using Agentic AI (fixes prior failure)
  - Agentic agents orchestrate end-to-end: generate UI in a no-code builder, configure data tables (Airtable/Google Sheets/Firebase) via their APIs, wire authentication (Firebase/Auth0), connect payments (Stripe/Stripe Connect), set up calendar integrations (Google Calendar API), and trigger deployment/publishing through the no-code builder and app-store APIs. No bespoke frontend/server code or manual orchestration needed.

- Highly profitable and easy to monetize
  - Clear revenue levers: commission on paid bookings (Stripe Connect), premium subscriptions for GMs (boosted listings, analytics), pay-for-featured placements, and a small marketplace fee for digital session materials. Payments handled entirely via existing Stripe APIs to avoid custom payment code.

- Attracts a niche but abundant audience
  - TTRPG (D&D, Pathfinder, etc.) communities are large and active but specialized: millions of players and thousands of active GMs seeking players or extra income. The niche is targeted (GMs/players) yet abundant enough for network effects and marketplace growth.

- Avoids heavy custom back-end systems (prefer existing APIs)
  - Use Airtable/Google Sheets or Firebase for data storage, Stripe for payments, Firebase/Auth0 for auth, Google Calendar API for scheduling, and no-code platforms for UI and wrappers. All orchestration done through public APIs — no custom server infrastructure needed.

- Avoids complex legal/regulatory exposure
  - The app facilitates hobby gaming bookings and communication (non-medical, non-financial advice, no sensitive personal data). Using Stripe Connect keeps payment/escrow responsibilities and KYC on Stripe; basic terms/consumer protections suffice. No handling of regulated goods/services.

Concise rollout path (how Agentic AI will build it)
- Agentic AI creates project spec, generates UI flows in a chosen no-code builder via its API.
- Agentic agents provision Airtable/Firebase schemas and populate templates.
- Agents wire authentication, Stripe Connect for payouts, and Google Calendar syncing.
- Agentic agents test flows and publish the PWA and native builds through the builder’s app-publishing APIs.
- Ongoing: agentic workflows handle onboarding templates, GM verification tips, and auto-generating listing copy/images for GMs.

This keeps scope tight, removes the need for custom frontends and orchestration code, relies on stable third-party APIs, targets a monetizable niche, and minimizes legal risk.

In [12]:
display(Markdown(qa_history))

TypeError: Markdown expects text, not [{'iteration': 1, 'score': 9, 'feedback': 'Failed: Cannot be built entirely using Agentic AI; still requires human-coded frontends and orchestration logic.', 'raw': '{"results": ["9", "Failed: Cannot be built entirely using Agentic AI; still requires human-coded frontends and orchestration logic."]}'}, {'iteration': 2, 'score': 10, 'feedback': 'Met all criteria', 'raw': '{"results": ["10", "Met all criteria"]}'}]